# 08 - Calculate consumption

#### Description

The goal of this notebook is to calculate the pest consumption at point buffer

It implements the following steps:

- calculate consumption of each species at the occurrence level
- calculate total consumption at the area of the point buffer

#### Inputs

- table with Daily Food Intake (DFI) for each species
- table with crop fraction for which birds provide service, created in step 06: `eBird_area_data_percent-step3.csv`
   

#### Outputs

- table with consumption at point level
- table with occurrence data and consumption per species, corrected for the effect of 

In [1]:
# import modules
import pandas as pd
import numpy as np
from pandas import DataFrame
import geopandas as gpd
from pyproj import Proj, CRS,transform
import matplotlib.pyplot as plt

## 1. Calculate consumption per crop area

Here, we multiply the consumption for each crop by the corresponding area of the crop.

Steps:

- add fraction of the area to the consumption table
- do calculations

In [2]:
# read occcurrence table

bird_data_name = pd.read_csv('../process_data/eBird_area_data_percent-step3.csv', low_memory = False)

In [3]:
# read DFI table
table_DFI = pd.read_csv('../process_data/table_DFI_2.csv', low_memory = False)

In [4]:
# select columns of occurrence table to merge with DFI

table = bird_data_name[['GLOBAL UNIQUE IDENTIFIER', 'OBSERVATION COUNT']]

In [5]:
# select column with area fraction of temporary crops to split in columns
table_t = bird_data_name[['GLOBAL UNIQUE IDENTIFIER', 'keep_t']]

In [6]:
# select column with area fraction of permanent crops to split in columns
table_p = bird_data_name[['GLOBAL UNIQUE IDENTIFIER', 'keep_p']]

In [7]:
table_t

,GLOBAL UNIQUE IDENTIFIER,keep_t
0,URN:CornellLabOfOrnithology:EBIRD:OBS1446126022,{'AVESA': 1.0}
1,URN:CornellLabOfOrnithology:EBIRD:OBS1446126014,{'AVESA': 1.0}
2,URN:CornellLabOfOrnithology:EBIRD:OBS1446126023,{'AVESA': 1.0}
3,URN:CornellLabOfOrnithology:EBIRD:OBS1446126018,{'AVESA': 1.0}
4,URN:CornellLabOfOrnithology:EBIRD:OBS1446126013,{'AVESA': 1.0}
...,...,...
78937,URN:CornellLabOfOrnithology:EBIRD:OBS394851198,{}
78938,URN:CornellLabOfOrnithology:EBIRD:OBS394868806,{}
78939,URN:CornellLabOfOrnithology:EBIRD:OBS395381323,{}
78940,URN:CornellLabOfOrnithology:EBIRD:OBS320781051,{}


In [8]:
# funtion to split crops and pests
def split_values(pair):
    if type(pair) != list: # wrong case, returns nan
        a = np.nan
        b = np.nan
    else:
        a = pair[0]
        b = pair[1]
    return a, b

In [9]:
# function to convert dict to list
def convert_to_list(valor):
    d = eval(valor) # if valor is not a dict, then use this
    l = [[k, v] for k, v in d.items()]
    #l = [[k, v] for k, v in valor.items()]
    return l

In [10]:
# apply convert column of dicts to list
table_t['keep_t_l'] = table_t.apply(lambda x: convert_to_list(x['keep_t']), axis=1)
table_p['keep_p_l'] = table_p.apply(lambda x: convert_to_list(x['keep_p']), axis=1)

<ipython-input-10-57d2a167ee8b>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  table_t['keep_t_l'] = table_t.apply(lambda x: convert_to_list(x['keep_t']), axis=1)
<ipython-input-10-57d2a167ee8b>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  table_p['keep_p_l'] = table_p.apply(lambda x: convert_to_list(x['keep_p']), axis=1)


In [11]:
# explode in several rows
table_t = table_t.explode('keep_t_l')
table_p = table_p.explode('keep_p_l')

In [12]:
# split list in two columns
table_t['eppo_crop'], table_t['crop_fract'] = zip(*table_t.apply(lambda x: split_values(x['keep_t_l']), axis=1))
table_p['eppo_crop'], table_p['crop_fract'] = zip(*table_p.apply(lambda x: split_values(x['keep_p_l']), axis=1))

In [13]:
table_p

,GLOBAL UNIQUE IDENTIFIER,keep_p,keep_p_l,eppo_crop,crop_fract
0,URN:CornellLabOfOrnithology:EBIRD:OBS1446126022,{'VITSS': 1.0},"[VITSS, 1.0]",VITSS,1.0
1,URN:CornellLabOfOrnithology:EBIRD:OBS1446126014,{'VITSS': 1.0},"[VITSS, 1.0]",VITSS,1.0
2,URN:CornellLabOfOrnithology:EBIRD:OBS1446126023,{'VITSS': 1.0},"[VITSS, 1.0]",VITSS,1.0
3,URN:CornellLabOfOrnithology:EBIRD:OBS1446126018,{'VITSS': 1.0},"[VITSS, 1.0]",VITSS,1.0
4,URN:CornellLabOfOrnithology:EBIRD:OBS1446126013,{'VITSS': 1.0},"[VITSS, 1.0]",VITSS,1.0
...,...,...,...,...,...
78937,URN:CornellLabOfOrnithology:EBIRD:OBS394851198,{},NaN,NaN,NaN
78938,URN:CornellLabOfOrnithology:EBIRD:OBS394868806,{},NaN,NaN,NaN
78939,URN:CornellLabOfOrnithology:EBIRD:OBS395381323,{},NaN,NaN,NaN
78940,URN:CornellLabOfOrnithology:EBIRD:OBS320781051,{},NaN,NaN,NaN


In [14]:
# merge DFI with occurrence values
table_DFI = pd.merge(table_DFI, table, on=['GLOBAL UNIQUE IDENTIFIER'])

In [15]:
# merge and concat tables to DFI
table_DFI_t = pd.merge(table_DFI, table_t, on=['GLOBAL UNIQUE IDENTIFIER', 'eppo_crop'])
table_DFI_p = pd.merge(table_DFI, table_p, on=['GLOBAL UNIQUE IDENTIFIER', 'eppo_crop'])
table_DFI = pd.concat([table_DFI_t, table_DFI_p])

In [16]:
table_DFI

,Unnamed: 0.1,Unnamed: 0,GLOBAL UNIQUE IDENTIFIER,Order2,Avibase.ID2,SAMPLING EVENT IDENTIFIER,SCIENTIFIC NAME_x,DEE,Annual_crops,Permanent_crops,...,crop_type,count_eat_ref,frac_DFI,consum_season,OBSERVATION COUNT,keep_t,keep_t_l,crop_fract,keep_p,keep_p_l
0,0,0,URN:CornellLabOfOrnithology:EBIRD:OBS1446126022,Passeriformes,AVIBASE-603194D3,S111628640,Thryomanes bewickii,194.086559,1.0,1.0,...,temp,4,9.882248,899.284564,1,{'AVESA': 1.0},"[AVESA, 1.0]",1.0,NaN,NaN
1,1,4137116,URN:CornellLabOfOrnithology:EBIRD:OBS1446126022,Passeriformes,AVIBASE-603194D3,S111628640,Thryomanes bewickii,194.086559,1.0,1.0,...,temp,4,14.282194,1299.679640,1,{'AVESA': 1.0},"[AVESA, 1.0]",1.0,NaN,NaN
2,2,9806702,URN:CornellLabOfOrnithology:EBIRD:OBS1446126022,Passeriformes,AVIBASE-603194D3,S111628640,Thryomanes bewickii,194.086559,1.0,1.0,...,temp,4,9.882248,899.284564,1,{'AVESA': 1.0},"[AVESA, 1.0]",1.0,NaN,NaN
3,3,12395944,URN:CornellLabOfOrnithology:EBIRD:OBS1446126022,Passeriformes,AVIBASE-603194D3,S111628640,Thryomanes bewickii,194.086559,1.0,1.0,...,temp,4,9.882248,899.284564,1,{'AVESA': 1.0},"[AVESA, 1.0]",1.0,NaN,NaN
4,8,5,URN:CornellLabOfOrnithology:EBIRD:OBS1446126023,Passeriformes,AVIBASE-151C2B3F,S111628640,Aphelocoma californica,4631.432930,0.0,1.0,...,temp,4,94.326920,8583.749764,1,{'AVESA': 1.0},"[AVESA, 1.0]",1.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1171090,3192436,10905600,URN:CornellLabOfOrnithology:EBIRD:OBS408081443,Strigiformes,AVIBASE-89F8B6F5,S29963429,Megascops kennicottii,8565.995447,0.0,1.0,...,perm,4,430.487943,39174.402775,1,NaN,NaN,1.0,{'OLVEU': 1.0},"[OLVEU, 1.0]"
1171091,3192437,10908027,URN:CornellLabOfOrnithology:EBIRD:OBS408081443,Strigiformes,AVIBASE-89F8B6F5,S29963429,Megascops kennicottii,8565.995447,0.0,1.0,...,perm,4,430.487943,39174.402775,1,NaN,NaN,1.0,{'OLVEU': 1.0},"[OLVEU, 1.0]"
1171092,3192438,10904448,URN:CornellLabOfOrnithology:EBIRD:OBS152618537,Strigiformes,AVIBASE-89F8B6F5,S10730732,Megascops kennicottii,8565.995447,0.0,1.0,...,perm,3,829.543004,75488.413382,1,NaN,NaN,1.0,{'PIAVE': 1.0},"[PIAVE, 1.0]"
1171093,3192439,10905661,URN:CornellLabOfOrnithology:EBIRD:OBS152618537,Strigiformes,AVIBASE-89F8B6F5,S10730732,Megascops kennicottii,8565.995447,0.0,1.0,...,perm,3,573.983923,52232.537033,1,NaN,NaN,1.0,{'PIAVE': 1.0},"[PIAVE, 1.0]"


In [17]:
# function to calculate final values of consumption

def calc_consumpt(crop_type, consum_season, Annual_crops, Permanent_crops, crop_fract, count):
    if crop_type == 'temp':
        consum = consum_season * Annual_crops * crop_fract * count
    if crop_type == 'perm':
        consum = consum_season * Permanent_crops * crop_fract * count

    return consum

In [18]:
# calculate total consumption
table_DFI['consum_total'] = table_DFI.apply(lambda x: calc_consumpt(x['crop_type'], x['consum_season'], \
                                                                    x['Annual_crops'], x['Permanent_crops'], \
                                                                    x['crop_fract'], x['OBSERVATION COUNT']), \
                                            axis = 1)


In [19]:
# identify as temporary or permanent crop 
table_DFI_temp = table_DFI[table_DFI['crop_type'] == 'temp']
table_DFI_perm = table_DFI[table_DFI['crop_type'] == 'perm']

In [20]:
# calculate total consumption per occurrence
table_DFI_temp['consum_total_t'] = table_DFI_temp.groupby(['GLOBAL UNIQUE IDENTIFIER'])['consum_total'].transform('sum')
table_DFI_perm['consum_total_p'] = table_DFI_perm.groupby(['GLOBAL UNIQUE IDENTIFIER'])['consum_total'].transform('sum')

<ipython-input-20-10337046dfa4>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  table_DFI_temp['consum_total_t'] = table_DFI_temp.groupby(['GLOBAL UNIQUE IDENTIFIER'])['consum_total'].transform('sum')
<ipython-input-20-10337046dfa4>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  table_DFI_perm['consum_total_p'] = table_DFI_perm.groupby(['GLOBAL UNIQUE IDENTIFIER'])['consum_total'].transform('sum')


In [21]:
table_DFI_temp

,Unnamed: 0.1,Unnamed: 0,GLOBAL UNIQUE IDENTIFIER,Order2,Avibase.ID2,SAMPLING EVENT IDENTIFIER,SCIENTIFIC NAME_x,DEE,Annual_crops,Permanent_crops,...,frac_DFI,consum_season,OBSERVATION COUNT,keep_t,keep_t_l,crop_fract,keep_p,keep_p_l,consum_total,consum_total_t
0,0,0,URN:CornellLabOfOrnithology:EBIRD:OBS1446126022,Passeriformes,AVIBASE-603194D3,S111628640,Thryomanes bewickii,194.086559,1.0,1.0,...,9.882248,899.284564,1,{'AVESA': 1.0},"[AVESA, 1.0]",1.000000,NaN,NaN,899.284564,3997.533333
1,1,4137116,URN:CornellLabOfOrnithology:EBIRD:OBS1446126022,Passeriformes,AVIBASE-603194D3,S111628640,Thryomanes bewickii,194.086559,1.0,1.0,...,14.282194,1299.679640,1,{'AVESA': 1.0},"[AVESA, 1.0]",1.000000,NaN,NaN,1299.679640,3997.533333
2,2,9806702,URN:CornellLabOfOrnithology:EBIRD:OBS1446126022,Passeriformes,AVIBASE-603194D3,S111628640,Thryomanes bewickii,194.086559,1.0,1.0,...,9.882248,899.284564,1,{'AVESA': 1.0},"[AVESA, 1.0]",1.000000,NaN,NaN,899.284564,3997.533333
3,3,12395944,URN:CornellLabOfOrnithology:EBIRD:OBS1446126022,Passeriformes,AVIBASE-603194D3,S111628640,Thryomanes bewickii,194.086559,1.0,1.0,...,9.882248,899.284564,1,{'AVESA': 1.0},"[AVESA, 1.0]",1.000000,NaN,NaN,899.284564,3997.533333
4,8,5,URN:CornellLabOfOrnithology:EBIRD:OBS1446126023,Passeriformes,AVIBASE-151C2B3F,S111628640,Aphelocoma californica,4631.432930,0.0,1.0,...,94.326920,8583.749764,1,{'AVESA': 1.0},"[AVESA, 1.0]",1.000000,NaN,NaN,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021341,3170087,17343010,URN:CornellLabOfOrnithology:EBIRD:OBS1414682541,Pelecaniformes,AVIBASE-6CCDAC53,S108928479,Bubulcus ibis,24832.088092,1.0,1.0,...,240.229937,21860.924235,1,"{'SORVU': 0.002070393374741201, 'ORYSA': 0.778...","[LYPES, 0.010351966873706004]",0.010352,NaN,NaN,226.303564,140927.508608
2021342,3170049,17284616,URN:CornellLabOfOrnithology:EBIRD:OBS1414682541,Pelecaniformes,AVIBASE-6CCDAC53,S108928479,Bubulcus ibis,24832.088092,1.0,1.0,...,522.173981,47517.832253,1,"{'SORVU': 0.002070393374741201, 'ORYSA': 0.778...","[TTLRI, 0.0010351966873706005]",0.001035,NaN,NaN,49.190303,140927.508608
2021343,3170061,17306006,URN:CornellLabOfOrnithology:EBIRD:OBS1414682541,Pelecaniformes,AVIBASE-6CCDAC53,S108928479,Bubulcus ibis,24832.088092,1.0,1.0,...,347.189277,31594.224201,1,"{'SORVU': 0.002070393374741201, 'ORYSA': 0.778...","[TTLRI, 0.0010351966873706005]",0.001035,NaN,NaN,32.706236,140927.508608
2021344,3170084,17331937,URN:CornellLabOfOrnithology:EBIRD:OBS1414682541,Pelecaniformes,AVIBASE-6CCDAC53,S108928479,Bubulcus ibis,24832.088092,1.0,1.0,...,240.229937,21860.924235,1,"{'SORVU': 0.002070393374741201, 'ORYSA': 0.778...","[TTLRI, 0.0010351966873706005]",0.001035,NaN,NaN,22.630356,140927.508608


In [22]:
# make smaller table, to better merge

occ_temp = table_DFI_temp.groupby(['GLOBAL UNIQUE IDENTIFIER']).first()
occ_temp.reset_index(inplace=True)

occ_perm = table_DFI_perm.groupby(['GLOBAL UNIQUE IDENTIFIER']).first()
occ_perm.reset_index(inplace=True)

In [23]:
# merge consumption to the occurrence table

bird_data_name_cons = pd.merge(bird_data_name, \
                               occ_temp[['GLOBAL UNIQUE IDENTIFIER', 'consum_total_t']], \
                               on='GLOBAL UNIQUE IDENTIFIER', how='left')

bird_data_name_cons = pd.merge(bird_data_name_cons, \
                               occ_perm[['GLOBAL UNIQUE IDENTIFIER', 'consum_total_p']], \
                               on='GLOBAL UNIQUE IDENTIFIER', how='left')

In [24]:
bird_data_name_cons['consum_total_t'] = bird_data_name_cons['consum_total_t'].fillna(0)
bird_data_name_cons['consum_total_p'] = bird_data_name_cons['consum_total_p'].fillna(0)

In [25]:
# save sample file, to check values
bird_data_name_cons.head(40).to_csv('../process_data/occ_consumption_head.csv')

In [26]:
bird_data_name_cons

,Unnamed: 0.2,Unnamed: 0.1,Unnamed: 0,GLOBAL UNIQUE IDENTIFIER,LAST EDITED DATE,TAXONOMIC ORDER_x,CATEGORY,TAXON CONCEPT ID_x,COMMON NAME,SCIENTIFIC NAME_x,...,fract_t_e,fract_p_e,consum_t,consum_t_ord,consum_p,consum_p_ord,keep_t,keep_p,consum_total_t,consum_total_p
0,0,0,0,URN:CornellLabOfOrnithology:EBIRD:OBS1446126022,2022-06-02 18:22:45.475308,26451,species,avibase-603194D3,Bewick's Wren,Thryomanes bewickii,...,{'AVESA': 1.0},{'VITSS': 1.0},"[['AVESA', 'DIABUH'], ['AVESA', 'HELIZE'], ['A...","[['AVESA', '1COLEO'], ['AVESA', '1LEPIO'], ['A...","[['VITSS', 'DIABVZ'], ['VITSS', 'PLAAST'], ['V...","[['VITSS', '1COLEO'], ['VITSS', '1LEPIO'], ['V...",{'AVESA': 1.0},{'VITSS': 1.0},3997.533333,3997.533333
1,1,1,1926,URN:CornellLabOfOrnithology:EBIRD:OBS1446126014,2022-06-02 18:22:45.475308,11494,species,avibase-20C2214E,American Kestrel,Falco sparverius,...,{'AVESA': 1.0},{'VITSS': 1.0},"[['AVESA', 'HETDZE'], ['AVESA', 'DITYDI'], ['A...","[['AVESA', '1RHABO'], ['AVESA', '1RHABO'], ['A...","[['VITSS', 'MELGET'], ['VITSS', 'DIABVZ'], ['V...","[['VITSS', '1RHABO'], ['VITSS', '1COLEO'], ['V...",{'AVESA': 1.0},{'VITSS': 1.0},0.000000,88366.927537
2,2,2,2608,URN:CornellLabOfOrnithology:EBIRD:OBS1446126023,2022-06-02 18:22:45.475308,20505,species,avibase-151C2B3F,California Scrub-Jay,Aphelocoma californica,...,{'AVESA': 1.0},{'VITSS': 1.0},"[['AVESA', 'DIABUH'], ['AVESA', 'HELIZE'], ['A...","[['AVESA', '1COLEO'], ['AVESA', '1LEPIO'], ['A...","[['VITSS', 'DIABVZ'], ['VITSS', 'PLAAST'], ['V...","[['VITSS', '1COLEO'], ['VITSS', '1LEPIO'], ['V...",{'AVESA': 1.0},{'VITSS': 1.0},0.000000,38156.805051
3,3,3,9726,URN:CornellLabOfOrnithology:EBIRD:OBS1446126018,2022-06-02 18:22:45.475308,11371,species,avibase-75600969,Northern Flicker,Colaptes auratus,...,{'AVESA': 1.0},{'VITSS': 1.0},"[['AVESA', 'DIABUH'], ['AVESA', 'HELIZE'], ['A...","[['AVESA', '1COLEO'], ['AVESA', '1LEPIO'], ['A...","[['VITSS', 'DIABVZ'], ['VITSS', 'PLAAST'], ['V...","[['VITSS', '1COLEO'], ['VITSS', '1LEPIO'], ['V...",{'AVESA': 1.0},{'VITSS': 1.0},191533.635201,191533.635201
4,4,4,10252,URN:CornellLabOfOrnithology:EBIRD:OBS1446126013,2022-06-02 18:22:45.475308,20762,species,avibase-9E9F2476,American Crow,Corvus brachyrhynchos,...,{'AVESA': 1.0},{'VITSS': 1.0},"[['AVESA', 'DIABUH'], ['AVESA', 'HELIZE'], ['A...","[['AVESA', '1COLEO'], ['AVESA', '1LEPIO'], ['A...","[['VITSS', 'DIABVZ'], ['VITSS', 'PLAAST'], ['V...","[['VITSS', '1COLEO'], ['VITSS', '1LEPIO'], ['V...",{'AVESA': 1.0},{'VITSS': 1.0},0.000000,253612.687788
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
78937,78937,78937,78917,URN:CornellLabOfOrnithology:EBIRD:OBS394851198,2021-03-31 02:38:15.410139,3433,species,avibase-9690689D,Lesser Nighthawk,Chordeiles acutipennis,...,"{'ZEAMX': 0.006578947368421052, 'ORYSA': 0.631...","{'PRNPS': 0.005055611729019211, 'VITSS': 0.002...",[],[],[],[],{},{},0.000000,0.000000
78938,78938,78938,78918,URN:CornellLabOfOrnithology:EBIRD:OBS394868806,2021-03-31 02:38:15.410139,3433,species,avibase-9690689D,Lesser Nighthawk,Chordeiles acutipennis,...,"{'ZEAMX': 0.006578947368421052, 'ORYSA': 0.631...","{'PRNPS': 0.005055611729019211, 'VITSS': 0.002...",[],[],[],[],{},{},0.000000,0.000000
78939,78939,78939,78919,URN:CornellLabOfOrnithology:EBIRD:OBS395381323,2021-03-31 02:38:15.410139,3433,species,avibase-9690689D,Lesser Nighthawk,Chordeiles acutipennis,...,"{'ZEAMX': 0.006578947368421052, 'ORYSA': 0.631...","{'PRNPS': 0.005055611729019211, 'VITSS': 0.002...",[],[],[],[],{},{},0.000000,0.000000
78940,78940,78940,78922,URN:CornellLabOfOrnithology:EBIRD:OBS320781051,2015-05-17 23:58:28,3433,species,avibase-9690689D,Lesser Nighthawk,Chordeiles acutipennis,...,{'MEDSA': 1.0},"{'MABSS': 0.043478260869565216, 'PRNDO': 0.956...",[],[],[],[],{},{},0.000000,0.000000


The next code calculates total occurrence consumption, summing temp and perm crop consumption

The calculated columns are:
- **consum_total_t_f** - consumption in the proportion of the area that are temporary crops. If bird trait implies that it only feeds in temporary crops, then it is the total consumption in crops
- **consum_total_p_t** -  consumption in the proportion of the area that are permanent crops. If bird trait implies that it only feeds in permanent crops, then it is the total consumption in crops 
- **consump_total_rest** - consumption in the proportion of the area that are available, but are not crops.
- **consump_grand_total** - total consumption in the three areas


In [27]:
# calculate total and correct proportion based on available area
def calc_total(annual, perm, temp_area, perm_area, avail_area, consum_t, consum_p):
     
    area_rest = avail_area - (temp_area + perm_area) # area available that is not crop
    
    # check if there is area available, and calculate fraction of area for crops and the rest
    if avail_area > 0:
        fract_t = temp_area / avail_area
        fract_p = perm_area / avail_area
        fract_r = area_rest / avail_area
    else:
        fract_t = 0
        fract_p = 0
        fract_r = 0
        

    if (annual == 1) and (perm == 1):                           # bird feeds in both temporary and permanent crops
        cons_t = (consum_t + consum_p) * fract_t
        cons_p = (consum_t + consum_p) * fract_p
        cons_r = (consum_t + consum_p) * fract_r
    elif (annual == 0) and (perm == 1):                         # bird feeds only in permanent crops
        cons_t = 0
        cons_p = (consum_t + consum_p) * (fract_p + fract_t)
        cons_r = (consum_t + consum_p) * fract_r
    elif (annual == 1) and (perm == 0):                         # bird feeds only in temporary crops
        cons_p = 0
        cons_t = (consum_t + consum_p) * (fract_p + fract_t)
        cons_r = (consum_t + consum_p) * fract_r
    else:
        cons_p = 0
        cons_t = 0
        cons_r = (consum_t + consum_p) * fract_r
        
    return cons_t, cons_p, cons_r, (cons_t + cons_p + cons_r)

In [28]:
# calculate total occurrence consumption, summing temp and perm crop consumption

bird_data_name_cons['consum_total_t_f'], bird_data_name_cons['consum_total_p_f'], \
bird_data_name_cons['consump_total_rest'], bird_data_name_cons['consump_grand_total'] = \
zip(*bird_data_name_cons.apply(lambda x: calc_total(x['Annual_crops'], x['Permanent_crops'], \
                                                    x['total_temp_area'], x['total_perm_area'], \
                                                    x['total_feed_area'], x['consum_total_t'], \
                                                    x['consum_total_p']), axis=1))

In [29]:
# calculate consumption per ha
bird_data_name_cons['consum_temp_ha'] = bird_data_name_cons['consum_total_t_f'] / bird_data_name_cons['total_temp_area']
bird_data_name_cons['consum_perm_ha'] = bird_data_name_cons['consum_total_p_f'] / bird_data_name_cons['total_perm_area']

In [30]:
bird_data_name_cons.to_csv('../process_data/bird_data_name_cons.csv')

### Checkpoint

In [31]:
# read file, here, if you already created and want to avoid running the previous steps
bird_data_name_cons = pd.read_csv('../process_data/bird_data_name_cons.csv', low_memory = False)

In [32]:
bird_data_name_cons.head()

,Unnamed: 0.3,Unnamed: 0.2,Unnamed: 0.1,Unnamed: 0,GLOBAL UNIQUE IDENTIFIER,LAST EDITED DATE,TAXONOMIC ORDER_x,CATEGORY,TAXON CONCEPT ID_x,COMMON NAME,...,keep_t,keep_p,consum_total_t,consum_total_p,consum_total_t_f,consum_total_p_f,consump_total_rest,consump_grand_total,consum_temp_ha,consum_perm_ha
0,0,0,0,0,URN:CornellLabOfOrnithology:EBIRD:OBS1446126022,2022-06-02 18:22:45.475308,26451,species,avibase-603194D3,Bewick's Wren,...,{'AVESA': 1.0},{'VITSS': 1.0},3997.533333,3997.533333,26.637538,30.442900,7937.986229,7995.066667,42.281806,42.281806
1,1,1,1,1926,URN:CornellLabOfOrnithology:EBIRD:OBS1446126014,2022-06-02 18:22:45.475308,11494,species,avibase-20C2214E,American Kestrel,...,{'AVESA': 1.0},{'VITSS': 1.0},0.000000,88366.927537,0.000000,630.891915,87736.035622,88366.927537,0.000000,876.238771
2,2,2,2,2608,URN:CornellLabOfOrnithology:EBIRD:OBS1446126023,2022-06-02 18:22:45.475308,20505,species,avibase-151C2B3F,California Scrub-Jay,...,{'AVESA': 1.0},{'VITSS': 1.0},0.000000,38156.805051,0.000000,272.418884,37884.386166,38156.805051,0.000000,378.359561
3,3,3,3,9726,URN:CornellLabOfOrnithology:EBIRD:OBS1446126018,2022-06-02 18:22:45.475308,11371,species,avibase-75600969,Northern Flicker,...,{'AVESA': 1.0},{'VITSS': 1.0},191533.635201,191533.635201,1276.283147,1458.609311,380332.377943,383067.270402,2025.846266,2025.846266
4,4,4,4,10252,URN:CornellLabOfOrnithology:EBIRD:OBS1446126013,2022-06-02 18:22:45.475308,20762,species,avibase-9E9F2476,American Crow,...,{'AVESA': 1.0},{'VITSS': 1.0},0.000000,253612.687788,0.000000,1810.656981,251802.030807,253612.687788,0.000000,2514.801362


In [33]:
bird_data_name_cons_cor = bird_data_name_cons.copy()

In [34]:
# create funtion to correct some species on the buffer
#
# consumption assumes that species feed in an area lower that a 1 km radius buffer. But some species with large 
# numbers are considered to feed in larger areas, up to 5 km radius. This function corrects that factor for the
# listed species
#
# species list includes all that have counts > 50

"""
species list

Agelaius tricolor
Petrochelidon pyrrhonota
Tachycineta bicolor
Euphagus cyanocephalus
Molothrus ater
Tachycineta thalassina
Corvus brachyrhynchos
Corvus corax
Hirundo rustica
Aeronautes saxatalis
Bubulcus ibis
Riparia riparia
Pica nuttalli
Quiscalus mexicanus
"""

# buff_1km / buff_5K = 0.04

sps_correct_buffer = ['Agelaius tricolor', 'Petrochelidon pyrrhonota', 'Tachycineta bicolor', 
                      'Euphagus cyanocephalus', 'Molothrus ater', 'Tachycineta thalassina', 'Corvus brachyrhynchos',
                      'Corvus corax', 'Hirundo rustica', 'Aeronautes saxatalis', 'Bubulcus ibis', 'Riparia riparia',
                      'Pica nuttalli', 'Quiscalus mexicanus']

def correct_buffer(sps, consum_total_t_f, consum_total_p_f, consum_total_rest, consum_grand_total, 
                  consum_temp_ha, consum_perm_ha):
    bfrac = 0.04
    if sps in sps_correct_buffer:
        t_f = consum_total_t_f*bfrac
        p_f = consum_total_p_f*bfrac
        rest = consum_total_rest*bfrac
        total = consum_grand_total*bfrac
        temp_ha = consum_temp_ha*bfrac
        perm_ha = consum_perm_ha*bfrac
        return t_f, p_f, rest, total, temp_ha, perm_ha
    else:
        return consum_total_t_f, consum_total_p_f, consum_total_rest, consum_grand_total, consum_temp_ha, consum_perm_ha
    

In [35]:
# create function to correct some species on the buffer

bird_data_name_cons_cor['consum_total_t_f'], bird_data_name_cons_cor['consum_total_p_f'], \
bird_data_name_cons_cor['consump_total_rest'], bird_data_name_cons_cor['consump_grand_total'], \
bird_data_name_cons_cor['consum_temp_ha'], bird_data_name_cons_cor['consum_perm_ha'] = \
zip(*bird_data_name_cons_cor.apply(lambda x: correct_buffer(x['Species2'], x['consum_total_t_f'], \
                                                    x['consum_total_p_f'], x['consump_total_rest'], \
                                                    x['consump_grand_total'], x['consum_temp_ha'], \
                                                    x['consum_perm_ha']), axis=1))

In [36]:
# write data to csv
bird_data_name_cons_cor.to_csv('../process_data/occ_consumption_cor.csv')

In [37]:
# write same data, but with less columns, to csv
bird_data_name_cons_cor[['GLOBAL UNIQUE IDENTIFIER','SCIENTIFIC NAME_x', 'OBSERVATION COUNT', \
                     'SAMPLING EVENT IDENTIFIER', 'OBSERVATION DATE', 'Annual_crops', 'Permanent_crops', \
                     'entr_f_area', 'total_area', 'total_crop_area', 'total_ncrop_area', 'total_wood_area', \
                     'total_nwood_area', 'total_feed_area', 'total_temp_area', 'total_perm_area', \
                     'consum_total_t', 'consum_total_p', 'consum_total_t_f', 'consum_total_p_f', \
                     'consump_total_rest', 'consump_grand_total', 'consum_temp_ha', \
                     'consum_perm_ha']].to_csv('../process_data/occ_consumption_simple_cor.csv')

In [38]:
bird_data_name_cons_cor.head(40).to_csv('../process_data/occ_consumption_cor_head.csv')

In [39]:
bird_data_name_cons.head(1000).to_csv('../process_data/occ_consumption1000.csv')

In [40]:
pd.set_option('display.max_columns', None)

## 2. Calculate values at point level

After performing all calculations at the occurrence level, we need to aggregate all occurrences of a point to obtain totals for the point.

In [41]:
pd.set_option('display.max_columns', None)

In [42]:
# read occurrence data with consumption values, corrected for large bands of birds
bird_data_name_cons_cor = pd.read_csv('../process_data/occ_consumption_cor.csv', low_memory=True)

In [43]:
# show number of rows and columns
bird_data_name_cons_cor.shape

(78942, 145)

In [44]:
# make a copy of the table
point_consumpt_tmp = bird_data_name_cons_cor.copy()

In [45]:
# calculate aggregations per event (point location)

# number of individuals (regardless of the species
point_consumpt_tmp['obs_count_total'] = \
    point_consumpt_tmp.groupby(['SAMPLING EVENT IDENTIFIER'])['OBSERVATION COUNT'].transform('sum')

# number of species that feed on annual crops
point_consumpt_tmp['count_sps_annual'] = \
    point_consumpt_tmp.groupby(['SAMPLING EVENT IDENTIFIER'])['Annual_crops'].transform('sum')

# number of species that feed on permanent crops
point_consumpt_tmp['count_sps_perm'] = \
    point_consumpt_tmp.groupby(['SAMPLING EVENT IDENTIFIER'])['Permanent_crops'].transform('sum')

# total consumption for temporary crops
point_consumpt_tmp['point_cons_tot_t'] = \
    point_consumpt_tmp.groupby(['SAMPLING EVENT IDENTIFIER'])['consum_total_t_f'].transform('sum')

# total consumption for permanent crops
point_consumpt_tmp['point_cons_tot_p'] = \
    point_consumpt_tmp.groupby(['SAMPLING EVENT IDENTIFIER'])['consum_total_p_f'].transform('sum')

# total consumption at the remaining available area for feeding
point_consumpt_tmp['point_cons_tot_rest'] = \
    point_consumpt_tmp.groupby(['SAMPLING EVENT IDENTIFIER'])['consump_total_rest'].transform('sum')

# total consumption
point_consumpt_tmp['point_cons_grand_tot'] = \
    point_consumpt_tmp.groupby(['SAMPLING EVENT IDENTIFIER'])['consump_grand_total'].transform('sum')

# total consumption temporary crops per hectare
point_consumpt_tmp['point_cons_temp_ha'] = \
    point_consumpt_tmp.groupby(['SAMPLING EVENT IDENTIFIER'])['consum_temp_ha'].transform('sum')

# total consumption permanent crops per hectare
point_consumpt_tmp['point_cons_perm_ha'] = \
    point_consumpt_tmp.groupby(['SAMPLING EVENT IDENTIFIER'])['consum_perm_ha'].transform('sum')

#### Calculate tree-dependency table

Tree dependence was classified for each species, based on the habitat density trait.

In [46]:
# read data on tree dependence
tree_dep = pd.read_csv('../external_data/AVONET/AvibaseApr2023_reclass_v3.csv')

In [47]:
tree_dep = tree_dep[['Species2', 'Trees_dependency']]

In [48]:
tree_dep.dropna(inplace=True)

In [49]:
tree_dep

,Species2,Trees_dependency
849,Aeronautes saxatalis,Low
897,Cypseloides niger,High
982,Chordeiles acutipennis,Medium
984,Chordeiles minor,Low
1017,Phalaenoptilus nuttallii,Low
...,...,...
10355,Pipilo maculatus,High
10438,Porzana carolina,Medium
10486,Eremophila alpestris,Low
10527,Corvus corax,Medium


In [50]:
# merge based on scientific name

point_consumpt_tmp = pd.merge(point_consumpt_tmp, tree_dep, 
                     left_on=['Species2'], right_on=['Species2'], how='left', indicator='exists')

In [51]:
# verify if there were no matches
test = point_consumpt_tmp[(point_consumpt_tmp['exists'] == 'left_only') | \
                           (point_consumpt_tmp['exists'] == 'right_only')]
test.shape

(0, 156)

In [52]:
# Tree dependence pivot table 
TreeDep = pd.pivot_table(point_consumpt_tmp, values = 'OBSERVATION COUNT', index = ['SAMPLING EVENT IDENTIFIER'], 
                   columns = ['Trees_dependency'], aggfunc='sum', fill_value=0)

# rename columns
TreeDep.rename(columns={"High": "Tree_high", "Medium": "Tree_medium", "Low": "Tree_low"}, inplace=True)

In [53]:
# calculate sum of rows, corresponds to the total number of birds at each sampling point
TreeDep['OBSERVATION COUNT'] = TreeDep.sum(axis=1)

In [54]:
# calculate fraction of each tree dependency class
TreeDep = TreeDep.iloc[:,:-1].div(TreeDep['OBSERVATION COUNT'], axis=0)

In [55]:
TreeDep

Trees_dependency,Tree_high,Tree_low,Tree_medium
SAMPLING EVENT IDENTIFIER,,,
S100923939,0.076923,0.615385,0.307692
S101565234,0.600000,0.300000,0.100000
S102616202,0.000000,0.000000,1.000000
S10320833,0.000000,0.466667,0.533333
S10325611,0.500000,0.250000,0.250000
...,...,...,...
S9898251,0.000000,0.000000,1.000000
S9898671,1.000000,0.000000,0.000000
S99077481,0.000000,0.800000,0.200000


#### Calculate guilds table

Guilds are defined from the combination of traits Trophic Level, Trophic Niche, Primary Lifestyle, ForagingNiche. We will calculate the consumption of each guild in temporary and permanent crops.

In [56]:
# ForagingNicheReclass (guilds) pivot table 
FNReclass = pd.pivot_table(point_consumpt_tmp, values = 'OBSERVATION COUNT', index = ['SAMPLING EVENT IDENTIFIER'], 
                   columns = ['ForagingNicheReclass'], aggfunc='sum', fill_value=0)


In [57]:
# calculate sum of rows, corresponds to the total number of birds at each sampling point
FNReclass['OBSERVATION COUNT'] = FNReclass.sum(axis=1)

In [58]:
# calculate fraction of each guild
FNReclass = FNReclass.iloc[:,:-1].div(FNReclass['OBSERVATION COUNT'], axis=0)

In [59]:
FNReclass

ForagingNicheReclass,Invertivore Aerial,Invertivore Generalist,Invertivore Generalist Generalist,Invertivore Insessorial,Invertivore Insessorial Generalist,Invertivore aerial,Invertivore bark,Invertivore glean arboreal,Invertivore ground,Invertivore sally air,Invertivore sally ground,Invertivore sally surface,Omnivore Generalist,Omnivore Insessorial,Omnivore Terrestrial
SAMPLING EVENT IDENTIFIER,,,,,,,,,,,,,,,
S100923939,0.0,0.0,0.0,0.0,0.0,0.153846,0.0,0.076923,0.0,0.0,0.0,0.0,0.000000,0.000000,0.769231
S101565234,0.0,0.0,0.0,0.0,0.2,0.000000,0.0,0.600000,0.1,0.0,0.0,0.0,0.000000,0.000000,0.100000
S102616202,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,1.000000
S10320833,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.133333,0.466667,0.400000
S10325611,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,1.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
S9898251,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,1.000000
S9898671,0.0,0.0,0.0,0.0,0.0,0.000000,1.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000
S99077481,0.0,0.0,0.0,0.0,0.2,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.800000,0.000000


#### Calculate number of individuals in each type of crop for each point

In [60]:
# determine number of individuals that use annual and permanent crops
df = point_consumpt_tmp.copy()

result = df[df['Annual_crops'] == 1].groupby('SAMPLING EVENT IDENTIFIER')['OBSERVATION COUNT'].sum()
df1 = pd.DataFrame(result, columns = ['OBSERVATION COUNT'])
df1.rename(columns={'OBSERVATION COUNT': 'count_ind_annual'}, inplace=True)

result2 = df[df['Permanent_crops'] == 1].groupby('SAMPLING EVENT IDENTIFIER')['OBSERVATION COUNT'].sum()
df2 = pd.DataFrame(result2, columns = ['OBSERVATION COUNT'])
df2.rename(columns={'OBSERVATION COUNT': 'count_ind_perm'}, inplace=True)

In [61]:
# merge with the main table
point_consumpt_tmp = pd.merge(point_consumpt_tmp, df1, on=['SAMPLING EVENT IDENTIFIER'], how='left')
point_consumpt_tmp = pd.merge(point_consumpt_tmp, df2, on=['SAMPLING EVENT IDENTIFIER'], how='left')

In [62]:
point_consumpt_tmp['count_ind_annual'] = point_consumpt_tmp['count_ind_annual'].fillna(0)
point_consumpt_tmp['count_ind_perm'] = point_consumpt_tmp['count_ind_perm'].fillna(0)

In [63]:
point_consumpt_tmp.head(2)

,Unnamed: 0.4,Unnamed: 0.3,Unnamed: 0.2,Unnamed: 0.1,Unnamed: 0,GLOBAL UNIQUE IDENTIFIER,LAST EDITED DATE,TAXONOMIC ORDER_x,CATEGORY,TAXON CONCEPT ID_x,COMMON NAME,SCIENTIFIC NAME_x,SUBSPECIES COMMON NAME,SUBSPECIES SCIENTIFIC NAME,EXOTIC CODE,OBSERVATION COUNT,BREEDING CODE,BREEDING CATEGORY,BEHAVIOR CODE,AGE/SEX,COUNTRY,COUNTRY CODE,STATE,STATE CODE,COUNTY,COUNTY CODE,LOCALITY_x,LOCALITY ID_x,LOCALITY TYPE,LATITUDE,LONGITUDE,OBSERVATION DATE,TIME OBSERVATIONS STARTED,OBSERVER ID,SAMPLING EVENT IDENTIFIER,PROTOCOL TYPE,PROTOCOL CODE,DURATION MINUTES,EFFORT DISTANCE KM,EFFORT AREA HA,NUMBER OBSERVERS,ALL SPECIES REPORTED,GROUP IDENTIFIER,APPROVED,REVIEWED,REASON,geometry_x,TAXON CONCEPT ID_y,SCIENTIFIC NAME_y,TAXONOMIC ORDER_y,ORDER1,Species2,Family2,Order2,Avibase.ID2,Total.individuals,Female,Male,Unknown,Complete.measures,Beak.Length_Culmen,Beak.Length_Nares,Beak.Width,Beak.Depth,Tarsus.Length,Wing.Length,Kipps.Distance,Secondary1,Hand-Wing.Index,Tail.Length,Mass,Mass.Source,Mass.Refs.Other,Inference,Traits.inferred,Reference.species,Habitat,Habitat.Density,Migration,Trophic.Level,Trophic.Niche,Primary.Lifestyle,ForagingNiche,ForagingNicheReclass,Annual_crops,Permanent_crops,Proportion_invertebrates_diet,_merge,NativeToValCal,LOCALITY_y,LOCALITY ID_y,geometry_y,stats_cover,area_buff,fract_c,fract_nc,fract_w,fract_nw,fract_f,fract_pn,fract_npn,fract_p,fract_t,entr_f_area,area,count_area,area_crop,count_crop,area_ncrop,count_ncrop,area_wood,count_wood,area_nwood,count_nwood,area_feed,count_feed,area_tcrop,count_tcrop,area_pcrop,count_pcrop,total_area,total_crop_area,total_ncrop_area,total_wood_area,total_nwood_area,total_feed_area,total_temp_area,total_perm_area,fract_c_e,fract_t_e,fract_p_e,consum_t,consum_t_ord,consum_p,consum_p_ord,keep_t,keep_p,consum_total_t,consum_total_p,consum_total_t_f,consum_total_p_f,consump_total_rest,consump_grand_total,consum_temp_ha,consum_perm_ha,obs_count_total,count_sps_annual,count_sps_perm,point_cons_tot_t,point_cons_tot_p,point_cons_tot_rest,point_cons_grand_tot,point_cons_temp_ha,point_cons_perm_ha,Trees_dependency,exists,count_ind_annual,count_ind_perm
0,0,0,0,0,0,URN:CornellLabOfOrnithology:EBIRD:OBS1446126022,2022-06-02 18:22:45.475308,26451,species,avibase-603194D3,Bewick's Wren,Thryomanes bewickii,NaN,NaN,NaN,1,NaN,NaN,NaN,NaN,United States,US,California,US-CA,Monterey,US-CA-053,"45491 Reliz Canyon Road, Greenfield, Californi...",L19350044,P,36.264001,-121.296414,2022-05-29,07:32:00,obsr836682,S111628640,Stationary,P21,6.0,NaN,NaN,2.0,1,G8497992,1,0,NaN,POINT (-121.2964144 36.2640014),AVIBASE-603194D3,Thryomanes bewickii,26451,Passeriformes,Thryomanes bewickii,Troglodytidae,Passeriformes,AVIBASE-603194D3,5,2,3,0,4.0,15.5,10.1,2.6,3.0,17.5,54.0,7.3,46.1,13.8,51.2,9.9,Dunning,NaN,NO,NaN,NaN,Shrubland,2,2.0,Carnivore,Invertivore,Generalist,Invertivore ground,Invertivore ground,1.0,1.0,1.0,both,1.0,"45491 Reliz Canyon Road, Greenfield, Californi...",L19350044,POLYGON ((-2224592.5060787485 1766274.99430329...,"{28: 7, 61: 2, 69: 8, 111: 1, 121: 59, 122: 8,...",195.3,"{28: 0.0032258064516129032, 69: 0.003686635944...","{61: 0.0009216589861751152, 111: 0.00046082949...","{142: 0.010599078341013824, 143: 0.01520737327...","{28: 0.0032258064516129032, 61: 0.000921658986...","{28: 0.0032258064516129032, 61: 0.000921658986...","{69: 0.003686635944700461, 142: 0.010599078341...","{28: 0.0032258064516129032, 61: 0.000921658986...",{69: 1.0},{28: 1.0},1.153677,"{28: 0.63, 61: 0.18, 69: 0.72, 111: 0.09, 121:...",13,"{28: 0.63, 69: 0.72}",2,"{61: 0.18, 111: 0.09, 121: 5.31, 122: 0.72, 13...",11,"{142: 2.07, 143: 2.97, 152: 132.21, 190: 0.72}",4,"{28: 0.63, 61: 0.18, 69: 0.72, 111: 0.09, 121:...",9,"{28: 0.63, 61: 0.18, 69: 0.72, 142: 2.07, 143:...",9,{28: 0.63},1,{69: 0.72},1,195.3,1.35,193.95,137.97,57.33,189.09,0.63,0.72,"{'AVESA': 0.0032258064516129032, 'VITSS': 0.00...",{'AVESA': 1.0},{'VITSS': 1.0},"[['AVESA', 'DIABUH'], ['AVESA', 'HELIZE'], ['A...","[['AVESA', '1COLEO'], ['AVESA', '1LEPIO'], [

In [64]:
point_consumpt_tmp.to_csv('../process_data/point_consumpt_tmp.csv')

In [65]:
# remove duplications, keep a row per sampling point
point_consumpt = point_consumpt_tmp.groupby(['SAMPLING EVENT IDENTIFIER']).first()

#### Combine consumption with tree dependency data

In [66]:
# merge points with tree dependency data
point_consumpt = pd.concat([point_consumpt, TreeDep], axis=1)

#### Combine consumption with guild data

In [67]:
# merge points with guild data
point_consumpt = pd.concat([point_consumpt, FNReclass], axis=1)

In [68]:
# make a copy of the table
point_consumpt_simple = point_consumpt.copy()

In [69]:
# remove columns that lost meaning because of the aggregation, or are duplications
point_consumpt_simple.drop(columns=['Unnamed: 0.4','Unnamed: 0.3','Unnamed: 0.2','Unnamed: 0.1',\
                                    'Unnamed: 0','GLOBAL UNIQUE IDENTIFIER','LAST EDITED DATE',\
                                    'TAXONOMIC ORDER_x','CATEGORY','TAXON CONCEPT ID_x','COMMON NAME',\
                                    'SCIENTIFIC NAME_x','SUBSPECIES COMMON NAME','SUBSPECIES SCIENTIFIC NAME',\
                                    'EXOTIC CODE','OBSERVATION COUNT','BREEDING CODE','BREEDING CATEGORY',\
                                    'BEHAVIOR CODE','AGE/SEX','EFFORT AREA HA','ALL SPECIES REPORTED',\
                                    'APPROVED','REVIEWED','REASON','TAXON CONCEPT ID_y','SCIENTIFIC NAME_y',\
                                    'TAXONOMIC ORDER_y','ORDER1','Species2','Family2','Order2','Avibase.ID2',\
                                    'Total.individuals','Female','Male','Unknown','Complete.measures',\
                                    'Beak.Length_Culmen','Beak.Length_Nares','Beak.Width','Beak.Depth',\
                                    'Tarsus.Length','Wing.Length','Kipps.Distance','Secondary1',\
                                    'Hand-Wing.Index','Tail.Length','Mass','Mass.Source','Mass.Refs.Other',\
                                    'Inference','Traits.inferred','Reference.species','Habitat',\
                                    'Habitat.Density','Migration','Trophic.Level','Trophic.Niche',\
                                    'Primary.Lifestyle','ForagingNiche','ForagingNicheReclass',\
                                    'Annual_crops','Permanent_crops','Proportion_invertebrates_diet',\
                                    '_merge','NativeToValCal','LOCALITY_y','LOCALITY ID_y', \
                                    'consum_total_t', 'consum_total_p','consum_total_t_f', \
                                    'consum_total_p_f', 'consump_total_rest', 'consump_grand_total'], inplace=True)

In [70]:
point_consumpt.head(2)

,Unnamed: 0.4,Unnamed: 0.3,Unnamed: 0.2,Unnamed: 0.1,Unnamed: 0,GLOBAL UNIQUE IDENTIFIER,LAST EDITED DATE,TAXONOMIC ORDER_x,CATEGORY,TAXON CONCEPT ID_x,COMMON NAME,SCIENTIFIC NAME_x,SUBSPECIES COMMON NAME,SUBSPECIES SCIENTIFIC NAME,EXOTIC CODE,OBSERVATION COUNT,BREEDING CODE,BREEDING CATEGORY,BEHAVIOR CODE,AGE/SEX,COUNTRY,COUNTRY CODE,STATE,STATE CODE,COUNTY,COUNTY CODE,LOCALITY_x,LOCALITY ID_x,LOCALITY TYPE,LATITUDE,LONGITUDE,OBSERVATION DATE,TIME OBSERVATIONS STARTED,OBSERVER ID,PROTOCOL TYPE,PROTOCOL CODE,DURATION MINUTES,EFFORT DISTANCE KM,EFFORT AREA HA,NUMBER OBSERVERS,ALL SPECIES REPORTED,GROUP IDENTIFIER,APPROVED,REVIEWED,REASON,geometry_x,TAXON CONCEPT ID_y,SCIENTIFIC NAME_y,TAXONOMIC ORDER_y,ORDER1,Species2,Family2,Order2,Avibase.ID2,Total.individuals,Female,Male,Unknown,Complete.measures,Beak.Length_Culmen,Beak.Length_Nares,Beak.Width,Beak.Depth,Tarsus.Length,Wing.Length,Kipps.Distance,Secondary1,Hand-Wing.Index,Tail.Length,Mass,Mass.Source,Mass.Refs.Other,Inference,Traits.inferred,Reference.species,Habitat,Habitat.Density,Migration,Trophic.Level,Trophic.Niche,Primary.Lifestyle,ForagingNiche,ForagingNicheReclass,Annual_crops,Permanent_crops,Proportion_invertebrates_diet,_merge,NativeToValCal,LOCALITY_y,LOCALITY ID_y,geometry_y,stats_cover,area_buff,fract_c,fract_nc,fract_w,fract_nw,fract_f,fract_pn,fract_npn,fract_p,fract_t,entr_f_area,area,count_area,area_crop,count_crop,area_ncrop,count_ncrop,area_wood,count_wood,area_nwood,count_nwood,area_feed,count_feed,area_tcrop,count_tcrop,area_pcrop,count_pcrop,total_area,total_crop_area,total_ncrop_area,total_wood_area,total_nwood_area,total_feed_area,total_temp_area,total_perm_area,fract_c_e,fract_t_e,fract_p_e,consum_t,consum_t_ord,consum_p,consum_p_ord,keep_t,keep_p,consum_total_t,consum_total_p,consum_total_t_f,consum_total_p_f,consump_total_rest,consump_grand_total,consum_temp_ha,consum_perm_ha,obs_count_total,count_sps_annual,count_sps_perm,point_cons_tot_t,point_cons_tot_p,point_cons_tot_rest,point_cons_grand_tot,point_cons_temp_ha,point_cons_perm_ha,Trees_dependency,exists,count_ind_annual,count_ind_perm,Tree_high,Tree_low,Tree_medium,Invertivore Aerial,Invertivore Generalist,Invertivore Generalist Generalist,Invertivore Insessorial,Invertivore Insessorial Generalist,Invertivore aerial,Invertivore bark,Invertivore glean arboreal,Invertivore ground,Invertivore sally air,Invertivore sally ground,Invertivore sally surface,Omnivore Generalist,Omnivore Insessorial,Omnivore Terrestrial
SAMPLING EVENT IDENTIFIER,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
S100923939,66419,66419,66419,66419,26701,URN:CornellLabOfOrnithology:EBIRD:OBS1320749038,2022-01-17 11:52:36.03516,23291,species,avibase-58C502EA,Barn Swallow,Hirundo rustica,None,None,NaN,2,None,None,None,None,United States,US,California,US-CA,Monterey,US-CA-053,Gonzales--Shell station,L2824373,P,36.523708,-121.465809,2014-04-05,10:20:00,obsr338000,Stationary,P21,10.0,NaN,NaN,34.0,1,G7732261,1,0,NaN,POINT (-121.4658093 36.5237082),AVIBASE-58C502EA,Hirundo rustica,23291,Passeriformes,Hirundo rustica,Hirundinidae,Passeriformes,AVIBASE-58C502EA,68,23,24,21,10.0,12.4,6.0,4.5,2.7,11.7,115.8,62.3,56.0,52.6,70.4,17.9,Dunning,NaN,NO,NaN,NaN,Human Modified,3,3.0,Carnivore,Invertivore,Aerial,Invertivore aerial,Invertivore aerial,1.0,1.0,1.0,both,1.0,Gonzales--Shell station,L2824373,POLYGON ((-2231436.4035407007 1798307.29557750...,"{1: 101, 2: 7, 3: 18, 4: 2, 6: 7, 12: 9, 21: 1...",313.29,"{1: 0.029014650962367137, 2: 0.002010916403332...","{37: 0.02585463947141626, 42: 0.00229819017523...","{142: 0.0005745475438092502, 143: 0.0002872737...","{1: 0.029014650962367137, 2: 0.002010916403332...","{1: 0.029014650962367137, 2: 0.002010916403332...","{66: 0.030738293593794888, 67: 0.0011490950876...","{1: 0.029014650962367137, 2: 0.002010916403332...","{66: 0.2482598607888631, 67: 0.009280742459396...","{1:

In [71]:
# remove columns that have detail not needed
point_consumpt_simple.drop(columns=['stats_cover', 'fract_c', 'fract_nc', 'fract_w', 'fract_nw', 'fract_f', 'fract_pn', 'fract_npn', \
                                    'fract_p', 'fract_t', 'area', 'area_crop', 'area_ncrop', 'area_wood', 'area_nwood', \
                                    'area_feed', 'area_tcrop', 'area_pcrop', 'fract_c_e', 'fract_t_e', 'fract_p_e', \
                                    'consum_t', 'consum_t_ord', 'consum_p', 'consum_p_ord', 'keep_t', 'keep_p'], inplace=True)

In [72]:
# show number of rows and columns
print(point_consumpt.shape)
print(point_consumpt_simple.shape)


(18783, 175)
(18783, 73)


In [73]:
# save tables to file
point_consumpt.to_csv('../process_data/point_consumpt.csv')
point_consumpt_simple.to_csv('../process_data/point_consumpt_simple.csv')